In [ ]:
from dep_tools.searchers import PystacSearcher
from dep_tools.loaders import OdcLoader

from odc.stac import configure_rio

from dep_tools.grids import PACIFIC_GRID_10
from src.utils import GeoMADSentinel2Processor

from dask.distributed import Client as DaskClient

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Set up rasterio
configure_rio(cloud_defaults=True, aws={"aws_unsigned": True})

In [ ]:
# Study site configuration
item_id = "63,20"  # West viti levu
# item_id = "65,22"  # Near the anti-meridian
# item_id = "66,22"  # Right of the anti-meridian
# item_id = "48,16"  # Vanuatu, middle of Grand Terre
# item_id = "89,16"  # Rarotonga

# item_id = "66,20"  # Area across am that is failing. TODO: find out why!
item_id = "65,20"  # To the west/left of the above tile

datetime = "2024"

# And get the study site
tile_index = tuple(int(i) for i in item_id.split(","))
geobox = PACIFIC_GRID_10.tile_geobox(tile_index)

# Load low-res if you want it faster
geobox = geobox.zoom_out(10)

geobox.explore()

In [ ]:
# Search for data
searcher = PystacSearcher(
    catalog="https://earth-search.aws.element84.com/v1",
    collections=["sentinel-2-c1-l2a"],
    datetime=datetime,   
)

items = searcher.search(area=geobox)
print(f"Found {len(items)} items")

In [ ]:
# Set up a data loader
loader = OdcLoader(
    # bands=["SCL", "B02", "B03", "B04", "B05", "B06", "B07", "B08", "B8A", "B11", "B12"],
    bands=["scl", "red", "blue", "green"],
    chunks=dict(time=1, x=3201, y=3201),
    groupby="solar_day",
    fail_on_error=False,
)

# Run the load process, which is lazy-loaded
input_data = loader.load(items, areas=geobox)
input_data

In [ ]:
old_filter = [("dilation", 3), ("erosion", 2)]
new_filter = [("erosion", 3), ("dilation", 6)]

# Set up a data processor
processor = GeoMADSentinel2Processor(
    geomad_options=dict(
        work_chunks=(600, 600),
        num_threads=10,
        maxiters=100,
        scale=1/10000,
        offset=-1000/10000
    ),
    min_timesteps=5,
    drop_vars = ["scl"],
)

# Do the processing using a local dask cluster
with DaskClient(n_workers=1, threads_per_worker=32, memory_limit="10GB"):
    output_data = processor.process(input_data)

output_data

In [ ]:
output_data.emad.dtype.name

In [ ]:
output_data.emad.odc.explore(robust=True)

In [ ]:
output_data[["red", "green", "blue"]].odc.explore(vmin=1000, vmax=3000)